# Full loans dataset schema

- **id**: unique id of the loan
- **member_id**: id of the member to took out the loan
- **loan_amount**: amount of loan the applicant received
- **funded_amount**: The total amount committed to the loan at the point in time 
- **funded_amount_inv**: The total amount committed by investors for that loan at that point in time 
- **term**: The number of monthly payments for the loan
- **int_rate**: Interest rate on the loan
- **instalment**: The monthly payment owned by the borrower
- **grade**: LC assigned loan grade
- **sub_grade**: LC assigned loan sub grade
- **employment_length**: Employment length in years.
- **home_ownership**: The home ownership status provided by the borrower
- **annual_inc**: The annual income of the borrower
- **verification_status**: Indicates whether the borrowers income was verified by the LC or the income source was verified
- **issue_date:** Issue date of the loan
- **loan_status**: Current status of the loan
- **payment_plan**: Indicates if a payment plan is in place for the loan. Indication borrower is struggling to pay.
- **purpose**: A category provided by the borrower for the loan request.
- **dti**: A ratio calculated using the borrowerâ€™s total monthly debt payments on the total debt obligations, excluding mortgage and the requested LC loan, divided by the borrowerâ€™s self-reported monthly income.
- **delinq_2yr**: The number of 30+ days past-due payment in the borrower's credit file for the past 2 years.
- **earliest_credit_line**: The month the borrower's earliest reported credit line was opened
- **inq_last_6mths**: The number of inquiries in past 6 months (excluding auto and mortgage inquiries)
- **mths_since_last_record**: The number of months since the last public record.
- **open_accounts**: The number of open credit lines in the borrower's credit file.
- **total_accounts**: The total number of credit lines currently in the borrower's credit file
- **out_prncp**: Remaining outstanding principal for total amount funded
- **out_prncp_inv**: Remaining outstanding principal for portion of total amount funded by investors
- **total_payment**: Payments received to date for total amount funded
- **total_rec_int**: Interest received to date
- **total_rec_late_fee**: Late fees received to date
- **recoveries**: post charge off gross recovery
- **collection_recovery_fee**: post charge off collection fee
- **last_payment_date**: Last month payment was received
- **last_payment_amount**: Last total payment amount received
- **next_payment_date**: Next scheduled payment date
- **last_credit_pull_date**: The most recent month LC pulled credit for this loan
- **collections_12_mths_ex_med**: Number of collections in 12 months excluding medical collections
- **mths_since_last_major_derog**: Months since most recent 90-day or worse rating
- **policy_code**: publicly available policy_code=1 new products not publicly available policy_code=2
- **application_type**: Indicates whether the loan is an individual application or a joint application with two co-borrowers

In [4]:
! python db_utils.py


In [4]:
if __name__ == "__main__":
    import yaml
    import pandas as pd
    import sqlalchemy

    class RDSDatabaseConnector:
        def get_creds(self):
            with open("credentials.yaml", "r") as file:
                result = dict(yaml.safe_load(file))
                return result

        def __init__(self):
            creds = self.get_creds()
            creds_string = f"postgresql://{creds['RDS_USER']}:{creds['RDS_PASSWORD']}@{creds['RDS_HOST']}:{creds['RDS_PORT']}/{creds['RDS_DATABASE']}"
            self.engine = sqlalchemy.create_engine(creds_string)

        def execute_query(self, query):
            with self.engine.connect() as conn:
                result = conn.execute(query)
                return result.fetchall()

        def extract_data(self):
            result = pd.DataFrame(self.execute_query("SELECT * FROM loan_payments"))
            return result

        def save_extracted_data(self):
            data = self.extract_data()
            data.to_csv("loan_data.csv")

    # # Usage
    # db_connector = RDSDatabaseConnector()
    # rows = db_connector.execute_query("SELECT * FROM loan_payments")
    # for row in rows:
    #     print(row)




In [5]:
data = pd.read_csv('loan_data.csv')

In [6]:
data.shape

(54231, 44)

# EDA FINDINGS 
- 44 Columns
- 54231 entries 
## COLUMNS WITH NULLS 
funded_amount
term
int_rate
employment_length
mths_since_last_delinq
mths_since_last_record
last_payment_date
next_payment_date
last_credit_pull_date
collections_12_mths_ex_med
mths_since_last_major_derog

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54231 entries, 0 to 54230
Data columns (total 44 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   54231 non-null  int64  
 1   id                           54231 non-null  int64  
 2   member_id                    54231 non-null  int64  
 3   loan_amount                  54231 non-null  int64  
 4   funded_amount                51224 non-null  float64
 5   funded_amount_inv            54231 non-null  float64
 6   term                         49459 non-null  object 
 7   int_rate                     49062 non-null  float64
 8   instalment                   54231 non-null  float64
 9   grade                        54231 non-null  object 
 10  sub_grade                    54231 non-null  object 
 11  employment_length            52113 non-null  object 
 12  home_ownership               54231 non-null  object 
 13  annual_inc      

In [15]:
null_cols = data.isnull()

In [8]:
null_cols = data.columns[data.isnull().any()].to_list()

In [9]:
null_cols

['funded_amount',
 'term',
 'int_rate',
 'employment_length',
 'mths_since_last_delinq',
 'mths_since_last_record',
 'last_payment_date',
 'next_payment_date',
 'last_credit_pull_date',
 'collections_12_mths_ex_med',
 'mths_since_last_major_derog']

In [27]:
for col in null_cols:
    print(col)
    

funded_amount
term
int_rate
employment_length
mths_since_last_delinq
mths_since_last_record
last_payment_date
next_payment_date
last_credit_pull_date
collections_12_mths_ex_med
mths_since_last_major_derog


In [31]:
data.sample(2)

,Unnamed: 0,id,member_id,loan_amount,funded_amount,funded_amount_inv,term,int_rate,instalment,grade,...,recoveries,collection_recovery_fee,last_payment_date,last_payment_amount,next_payment_date,last_credit_pull_date,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type
3466,3466,37751530,40514487,28800,28800.0,28800.0,60 months,19.24,750.90,E,...,0.0,0.0,Jan-2022,750.90,Feb-2022,Jan-2022,0.0,59.0,1,INDIVIDUAL
30102,30102,6737905,8340213,2000,2000.0,2000.0,36 months,13.05,67.44,B,...,0.0,0.0,Sep-2019,2021.75,NaN,Apr-2020,0.0,NaN,1,INDIVIDUAL


In [32]:
data.columns

Index(['Unnamed: 0', 'id', 'member_id', 'loan_amount', 'funded_amount',
       'funded_amount_inv', 'term', 'int_rate', 'instalment', 'grade',
       'sub_grade', 'employment_length', 'home_ownership', 'annual_inc',
       'verification_status', 'issue_date', 'loan_status', 'payment_plan',
       'purpose', 'dti', 'delinq_2yrs', 'earliest_credit_line',
       'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record',
       'open_accounts', 'total_accounts', 'out_prncp', 'out_prncp_inv',
       'total_payment', 'total_payment_inv', 'total_rec_prncp',
       'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'collection_recovery_fee', 'last_payment_date', 'last_payment_amount',
       'next_payment_date', 'last_credit_pull_date',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'policy_code', 'application_type'],
      dtype='object')

In [10]:
cols_numeric_required = ['loan_amount', 'funded_amount', 'funded_amount_inv', 'int_rate', 'open_accounts', 'total_accounts','total_payment', 'total_payment_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries',
'collection_recovery_fee', 'collections_12_mths_ex_med', 'mths_since_last_major_derog']

In [11]:
data[cols_numeric_required] = pd.to_numeric(data[cols_numeric_required])

TypeError: arg must be a list, tuple, 1-d array, or Series

In [40]:
data[cols_numeric_required].sample(5)

,loan_amount,funded_amount,funded_amount_inv,int_rate,open_accounts,total_accounts,total_payment,total_payment_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,collections_12_mths_ex_med,mths_since_last_major_derog
23232,12000,12000.0,12000.0,13.67,5,9,11021.500000,11021.50,8526.90,2494.60,0.0,0.00,0.00,0.0,NaN
13319,2000,2000.0,2000.0,14.30,9,12,2396.468209,2396.47,2000.00,396.47,0.0,0.00,0.00,0.0,NaN
628,8000,8000.0,8000.0,11.99,7,12,8799.930000,8799.93,8000.00,799.93,0.0,0.00,0.00,0.0,NaN
27249,11625,11625.0,11625.0,15.61,18,39,7006.960000,7006.96,3784.47,3222.49,0.0,0.00,0.00,0.0,NaN
45486,2500,2500.0,2200.0,7.14,6,24,1974.790000,1737.39,1673.49,254.26,0.0,47.04,0.67,0.0,NaN


In [12]:
for col in cols_numeric_required:
    data[col] = pd.to_numeric(data[col], errors='coerce')

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54231 entries, 0 to 54230
Data columns (total 44 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   54231 non-null  int64  
 1   id                           54231 non-null  int64  
 2   member_id                    54231 non-null  int64  
 3   loan_amount                  54231 non-null  int64  
 4   funded_amount                51224 non-null  float64
 5   funded_amount_inv            54231 non-null  float64
 6   term                         49459 non-null  object 
 7   int_rate                     49062 non-null  float64
 8   instalment                   54231 non-null  float64
 9   grade                        54231 non-null  object 
 10  sub_grade                    54231 non-null  object 
 11  employment_length            52113 non-null  object 
 12  home_ownership               54231 non-null  object 
 13  annual_inc      

In [53]:
data.describe()

,Unnamed: 0,id,member_id,loan_amount,funded_amount,funded_amount_inv,int_rate,instalment,annual_inc,dti,...,total_payment_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_payment_amount,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code
count,54231.000000,5.423100e+04,5.423100e+04,54231.000000,51224.000000,54231.000000,49062.000000,54231.000000,5.423100e+04,54231.000000,...,54231.000000,54231.000000,54231.000000,54231.000000,54231.000000,54231.000000,54231.000000,54180.000000,7499.000000,54231.0
mean,27115.000000,7.621797e+06,8.655350e+06,13333.076100,13229.509117,12952.622979,13.507328,400.013953,7.222085e+04,15.867087,...,11788.946618,9407.048589,2577.757101,0.901512,93.501288,10.859057,3130.706393,0.004208,42.253634,1.0
std,15655.285561,9.571362e+06,1.031281e+07,8082.196709,8019.017599,8099.473527,4.392893,238.920012,5.158934e+04,7.623124,...,8363.508506,6958.124264,2581.657345,6.215792,630.843636,120.193950,5323.801675,0.070990,21.052360,0.0
min,0.000000,5.552100e+04,7.069400e+04,500.000000,500.000000,0.000000,5.420000,15.670000,3.300000e+03,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
25%,13557.500000,7.594330e+05,9.587720e+05,7000.000000,7000.000000,6700.000000,10.370000,224.205000,4.500000e+04,10.200000,...,5658.815000,4264.350000,889.560000,0.000000,0.000000,0.000000,289.790000,0.000000,26.000000,1.0
50%,27115.000000,7.084590e+06,8.709873e+06,12000.000000,12000.000000,11300.000000,13.160000,347.150000,6.100000e+04,15.600000,...,9835.830000,7644.920000,1734.640000,0.000000,0.000000,0.000000,562.670000,0.000000,42.000000,1.0
75%,40672.500000,8.860616e+06,1.052714e+07,18000.000000,18000.000000,18000.000000,16.200000,527.550000,8.600000e+04,21.260000,...,15978.200000,12505.625000,3323.765000,0.000000,0.000000,0.000000,3738.120000,0.000000,59.000000,1.0
max,54230.000000,3.867612e+07,4.146185e+07,35000.000000,35000.000000,35000.000000,26.060000,1407.010000,2.039784e+06,39.910000,...,55061.000000,35000.020000,23062.450000,358.680000,27750.000000,7002.190000,36115.200000,4.000000,146.000000,1.0


In [54]:
import transformations

In [14]:
datecols= [col for col in data.columns if 'date' in col]

In [15]:
transformations.Transforms.cols_to_datetime(datecols, data)

NameError: name 'transformations' is not defined

In [62]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54231 entries, 0 to 54230
Data columns (total 44 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   Unnamed: 0                   54231 non-null  int64         
 1   id                           54231 non-null  int64         
 2   member_id                    54231 non-null  int64         
 3   loan_amount                  54231 non-null  int64         
 4   funded_amount                51224 non-null  float64       
 5   funded_amount_inv            54231 non-null  float64       
 6   term                         49459 non-null  object        
 7   int_rate                     49062 non-null  float64       
 8   instalment                   54231 non-null  float64       
 9   grade                        54231 non-null  object        
 10  sub_grade                    54231 non-null  object        
 11  employment_length            52113 non-nu

In [16]:
data.drop("Unnamed: 0", axis=1, inplace=True)

In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54231 entries, 0 to 54230
Data columns (total 43 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   id                           54231 non-null  int64  
 1   member_id                    54231 non-null  int64  
 2   loan_amount                  54231 non-null  int64  
 3   funded_amount                51224 non-null  float64
 4   funded_amount_inv            54231 non-null  float64
 5   term                         49459 non-null  object 
 6   int_rate                     49062 non-null  float64
 7   instalment                   54231 non-null  float64
 8   grade                        54231 non-null  object 
 9   sub_grade                    54231 non-null  object 
 10  employment_length            52113 non-null  object 
 11  home_ownership               54231 non-null  object 
 12  annual_inc                   54231 non-null  float64
 13  verification_sta

In [74]:
data.groupby('loan_status').describe()

id                \
                                                      count          mean   
loan_status                                                                 
Charged Off                                          5571.0  5.288030e+06   
Current                                             19268.0  1.299020e+07   
Default                                                52.0  9.624598e+06   
Does not meet the credit policy. Status:Charged...    368.0  3.844220e+05   
Does not meet the credit policy. Status:Fully Paid    984.0  4.009459e+05   
Fully Paid                                          27037.0  4.454480e+06   
In Grace Period                                       265.0  1.106917e+07   
Late (16-30 days)                                     106.0  1.620923e+07   
Late (31-120 days)                                    580.0  1.286067e+07   

                                                                          \
                                                         min         25%   
loan_status                                                                
Charged Off                                          61419.0   700027.00   
Current                                             356706.0  7306748.00   
Default                                             609371.0  7075521.50   
Does not meet the credit policy. Status:Charged...   56413.0   251555.00   
Does not meet the credit policy. Status:Fully Paid   55521.0   306595.25   
Fully Paid                                           55742.0   606548.00   
In Grace Period                                     645021.0  7068593.00   
Late (16-30 days)                                   624521.0  7627979.75   
Late (31-120 days)                                  560150.0  7364797.00   

                                                                            \
                                                          50%          75%   
loan_status                                                                  
Charged Off                                         6704799.0   8194655.50   
Current                                             8589107.5   9785256.00   
Default                                             7947327.5   9058834.75   
Does not meet the credit policy. Status:Charged...   388717.0    518048.25   
Does not meet the credit policy. Status:Fully Paid   420008.5    504541.00   
Fully Paid                                           972601.0   7689656.00   
In Grace Period                                     7924777.0   9196455.00   
Late (16-30 days)                                   8943168.5  37570062.50   
Late (31-120 days)                                  8612383.5   9775957.00   

                                                                              \
                                                           max           std   
loan_status                                                                    
Charged Off                                         38656067.0  6.465143e+06   
Current                                             38676116.0  1.165641e+07   
Default                                             38556083.0  8.726418e+06   
Does not meet the credit policy. Status:Charged...    640893.0  1.556542e+05   
Does not meet the credit policy. Status:Fully Paid    641638.0  1.420494e+05   
Fully Paid                                          38656154.0  6.191486e+06   
In Grace Period                                     38635085.0  1.010332e+07   
Late (16-30 days)                                   38625326.0  1.397001e+07   
Late (31-120 days)                                  38655869.0  1.138809e+07   

                                                   member_id                \
                                                       count          mean   
loan_status                                                                  
Charged Off                                           5571.0  6.098112e+06   
Current  

In [18]:
a = [col for col in data.columns if 'status' in col]

In [76]:
a

['verification_status', 'loan_status']

In [77]:
data.a.value_counts()

AttributeError: 'DataFrame' object has no attribute 'a'

In [19]:
data[a].value_counts()

verification_status  loan_status                                        
Not Verified         Fully Paid                                             10086
Verified             Fully Paid                                              9521
                     Current                                                 8125
Source Verified      Fully Paid                                              7430
                     Current                                                 6413
Not Verified         Current                                                 4730
Verified             Charged Off                                             2358
Not Verified         Charged Off                                             1632
Source Verified      Charged Off                                             1581
Not Verified         Does not meet the credit policy. Status:Fully Paid       647
Verified             Late (31-120 days)                                       272
Not Verified         Does

In [81]:
data['annual_inc'].dtype

dtype('float64')

In [20]:
tt = [xol for xol in data.columns if data[xol].dtype == 'O']

In [83]:
tt

['term',
 'grade',
 'sub_grade',
 'employment_length',
 'home_ownership',
 'verification_status',
 'loan_status',
 'payment_plan',
 'purpose',
 'earliest_credit_line',
 'application_type']

In [85]:
data[tt]

,term,grade,sub_grade,employment_length,home_ownership,verification_status,loan_status,payment_plan,purpose,earliest_credit_line,application_type
0,36 months,A,A4,5 years,MORTGAGE,Not Verified,Current,n,credit_card,Oct-1987,INDIVIDUAL
1,36 months,A,A3,9 years,RENT,Not Verified,Current,n,credit_card,Sep-2001,INDIVIDUAL
2,36 months,A,A4,8 years,MORTGAGE,Source Verified,Fully Paid,n,credit_card,Sep-1998,INDIVIDUAL
3,36 months,C,C4,1 year,RENT,Source Verified,Fully Paid,n,debt_consolidation,Jun-2008,INDIVIDUAL
4,36 months,A,A1,10+ years,MORTGAGE,Verified,Current,n,debt_consolidation,Apr-2002,INDIVIDUAL
...,...,...,...,...,...,...,...,...,...,...,...
54226,36 months,B,B2,1 year,MORTGAGE,Not Verified,Fully Paid,n,other,Apr-2003,INDIVIDUAL
54227,36 months,C,C2,< 1 year,RENT,Not Verified,Fully Paid,n,other,Jan-1999,INDIVIDUAL
54228,36 months,A,A2,10+ years,MORTGAGE,Not Verified,Does not meet the credit policy. Status:Fully ...,n,debt_consolidation,Feb-1984,INDIVIDUAL
54229,36 months,A,A2,4 years,RENT,Not Verified,Fully Paid,n,house,Mar-1995,INDIVIDUAL


In [89]:
for t in tt:
   print(f' column name: {t}  value counts:  {data[t].value_counts()}')

 column name: term  value counts:  term
36 months    35845
60 months    13614
Name: count, dtype: int64
 column name: grade  value counts:  grade
B    16369
C    13600
A     9818
D     8157
E     4072
F     1694
G      521
Name: count, dtype: int64
 column name: sub_grade  value counts:  sub_grade
B3    3641
B4    3542
B2    3241
C1    3018
B5    3016
C2    2962
B1    2929
C3    2794
A5    2789
A4    2645
C4    2512
C5    2314
D1    1838
D2    1809
A3    1752
D3    1647
D4    1509
A2    1481
D5    1354
A1    1151
E1     989
E2     985
E3     817
E4     658
E5     623
F1     486
F2     390
F3     326
F4     287
F5     205
G1     156
G2     133
G3      87
G4      74
G5      71
Name: count, dtype: int64
 column name: employment_length  value counts:  employment_length
10+ years    15907
2 years       5066
< 1 year      5034
3 years       4589
5 years       3836
1 year        3708
4 years       3482
6 years       3059
7 years       2964
8 years       2537
9 years       1931
Name: count, dt

In [21]:
cateogric_columns = ['term',
 'grade',
 'sub_grade',
 'home_ownership',
 'verification_status',
 'loan_status',
 'payment_plan',
 'purpose',
 'application_type']

In [92]:
cateogric_columns


['term',
 'grade',
 'sub_grade',
 'home_ownership',
 'verification_status',
 'loan_status',
 'payment_plan',
 'purpose',
 'application_type']

In [22]:
for col in cateogric_columns:
    data[col] = data[col].astype('category')

In [23]:
data['home_ownership'].dtype

CategoricalDtype(categories=['MORTGAGE', 'NONE', 'OTHER', 'OWN', 'RENT'], ordered=False, categories_dtype=object)

In [24]:
data['earliest_credit_line'] = pd.to_datetime(data['earliest_credit_line'], errors='coerce')

/var/folders/4n/18wn5fm9177dpn28hhj4f8w00000gn/T/ipykernel_37755/72792868.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['earliest_credit_line'] = pd.to_datetime(data['earliest_credit_line'], errors='coerce')


# Now the data types look correct.

In [25]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54231 entries, 0 to 54230
Data columns (total 43 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   id                           54231 non-null  int64         
 1   member_id                    54231 non-null  int64         
 2   loan_amount                  54231 non-null  int64         
 3   funded_amount                51224 non-null  float64       
 4   funded_amount_inv            54231 non-null  float64       
 5   term                         49459 non-null  category      
 6   int_rate                     49062 non-null  float64       
 7   instalment                   54231 non-null  float64       
 8   grade                        54231 non-null  category      
 9   sub_grade                    54231 non-null  category      
 10  employment_length            52113 non-null  object        
 11  home_ownership               54231 non-nu

In [26]:
datecols

['issue_date',
 'last_payment_date',
 'next_payment_date',
 'last_credit_pull_date']

In [28]:
for column in datecols:
    data[column] = pd.to_datetime(data[column])

/var/folders/4n/18wn5fm9177dpn28hhj4f8w00000gn/T/ipykernel_37755/4027835274.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data[column] = pd.to_datetime(data[column])
/var/folders/4n/18wn5fm9177dpn28hhj4f8w00000gn/T/ipykernel_37755/4027835274.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data[column] = pd.to_datetime(data[column])
/var/folders/4n/18wn5fm9177dpn28hhj4f8w00000gn/T/ipykernel_37755/4027835274.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data[column] = pd.to_datetime(data[column])
/var/folders/4n/18wn5fm9177dpn28hhj4f8w00000gn/T/ipykernel_37755

In [29]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54231 entries, 0 to 54230
Data columns (total 43 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   id                           54231 non-null  int64         
 1   member_id                    54231 non-null  int64         
 2   loan_amount                  54231 non-null  int64         
 3   funded_amount                51224 non-null  float64       
 4   funded_amount_inv            54231 non-null  float64       
 5   term                         49459 non-null  category      
 6   int_rate                     49062 non-null  float64       
 7   instalment                   54231 non-null  float64       
 8   grade                        54231 non-null  category      
 9   sub_grade                    54231 non-null  category      
 10  employment_length            52113 non-null  object        
 11  home_ownership               54231 non-nu

In [35]:
data.employment_length.dtype

dtype('O')

In [42]:
data['employment_length'] = data['employment_length'].fillna(0)
data['employment_length'] = data['employment_length'].str.extract('(\d+)')
data['employment_length'] = data['employment_length'].astype(float).astype('Int64')  # 'Int64' (capital I) allows for NaN values in integer columns

In [44]:
data.employment_length.rename('employment_length_years', inplace=True)

0         5
1         9
2         8
3         1
4        10
         ..
54226     1
54227     1
54228    10
54229     4
54230     9
Name: employment_length_years, Length: 54231, dtype: Int64

In [45]:
data.employment_length.dtype

Int64Dtype()

In [46]:
data.to_csv('loan_data.csv')

In [47]:
df = pd.read_csv('loan_data.csv')

In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54231 entries, 0 to 54230
Data columns (total 44 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   54231 non-null  int64  
 1   id                           54231 non-null  int64  
 2   member_id                    54231 non-null  int64  
 3   loan_amount                  54231 non-null  int64  
 4   funded_amount                51224 non-null  float64
 5   funded_amount_inv            54231 non-null  float64
 6   term                         49459 non-null  object 
 7   int_rate                     49062 non-null  float64
 8   instalment                   54231 non-null  float64
 9   grade                        54231 non-null  object 
 10  sub_grade                    54231 non-null  object 
 11  employment_length            52113 non-null  float64
 12  home_ownership               54231 non-null  object 
 13  annual_inc      

In [49]:
df.term

0        36 months
1        36 months
2        36 months
3        36 months
4        36 months
           ...    
54226    36 months
54227    36 months
54228    36 months
54229    36 months
54230    36 months
Name: term, Length: 54231, dtype: object

In [50]:
df.term = df.term.str.extract('(\d+)')

In [52]:
df.term = df.term.rename('term_months', inplace = True)

In [69]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54231 entries, 0 to 54230
Data columns (total 44 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   Unnamed: 0                   54231 non-null  int64         
 1   id                           54231 non-null  int64         
 2   member_id                    54231 non-null  int64         
 3   loan_amount                  54231 non-null  int64         
 4   funded_amount                51224 non-null  float64       
 5   funded_amount_inv            54231 non-null  float64       
 6   term                         49459 non-null  category      
 7   int_rate                     49062 non-null  float64       
 8   instalment                   54231 non-null  float64       
 9   grade                        54231 non-null  category      
 10  sub_grade                    54231 non-null  category      
 11  employment_length            52113 non-nu

In [67]:
obj = [col for col in df.columns if df[col].dtype == 'O']

In [68]:
for o in obj:
    df[o] = df[o].astype('category')

In [62]:
datecols = ['issue_date',
 'last_payment_date',
 'next_payment_date',
 'last_credit_pull_date',
 'earliest_credit_line']

In [63]:
for col in datecols:
    df[col] = pd.to_datetime(df[col])

In [70]:
df.sample(2)

,Unnamed: 0,id,member_id,loan_amount,funded_amount,funded_amount_inv,term,int_rate,instalment,grade,...,recoveries,collection_recovery_fee,last_payment_date,last_payment_amount,next_payment_date,last_credit_pull_date,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type
37732,37732,856631,1069013,4000,4000.0,4000.0,60,16.89,99.18,D,...,0.0,0.0,2021-06-01,1455.73,NaT,2022-01-01,0.0,NaN,1,INDIVIDUAL
40032,40032,802377,982357,25000,25000.0,25000.0,60,19.69,658.05,E,...,0.0,0.0,2018-08-01,4842.52,NaT,2018-08-01,0.0,NaN,1,INDIVIDUAL


In [71]:
df.to_csv('loan_data.csv') # Overwritten with clean data

Describe all columns in the DataFrame to check their data types
Extract statistical values: median, standard deviation and mean from the columns and the DataFrame
Count distinct values in categorical columns
Print out the shape of the DataFrame
Generate a count/percentage count of NULL values in each column
Any other methods you may find useful



In [78]:
df.drop("Unnamed: 0", axis=1, inplace=True )

In [79]:
for column in df.columns:
    print(f'description of column, {column}: {df[column].describe()}')

description of column, id: count    5.423100e+04
mean     7.621797e+06
std      9.571362e+06
min      5.552100e+04
25%      7.594330e+05
50%      7.084590e+06
75%      8.860616e+06
max      3.867612e+07
Name: id, dtype: float64
description of column, member_id: count    5.423100e+04
mean     8.655350e+06
std      1.031281e+07
min      7.069400e+04
25%      9.587720e+05
50%      8.709873e+06
75%      1.052714e+07
max      4.146185e+07
Name: member_id, dtype: float64
description of column, loan_amount: count    54231.000000
mean     13333.076100
std       8082.196709
min        500.000000
25%       7000.000000
50%      12000.000000
75%      18000.000000
max      35000.000000
Name: loan_amount, dtype: float64
description of column, funded_amount: count    51224.000000
mean     13229.509117
std       8019.017599
min        500.000000
25%       7000.000000
50%      12000.000000
75%      18000.000000
max      35000.000000
Name: funded_amount, dtype: float64
description of column, funded_amou

## percentage of nulls in each column: 

In [83]:
for column in df.columns:
    print(f"the percentage of nulls for column {column}: is {df[column].isnull().sum()/ 54231* 100.0}")


the percentage of nulls for column id: is 0.0
the percentage of nulls for column member_id: is 0.0
the percentage of nulls for column loan_amount: is 0.0
the percentage of nulls for column funded_amount: is 5.544799100145673
the percentage of nulls for column funded_amount_inv: is 0.0
the percentage of nulls for column term: is 8.799395179878667
the percentage of nulls for column int_rate: is 9.53144880234552
the percentage of nulls for column instalment: is 0.0
the percentage of nulls for column grade: is 0.0
the percentage of nulls for column sub_grade: is 0.0
the percentage of nulls for column employment_length: is 3.905515295679593
the percentage of nulls for column home_ownership: is 0.0
the percentage of nulls for column annual_inc: is 0.0
the percentage of nulls for column verification_status: is 0.0
the percentage of nulls for column issue_date: is 0.0
the percentage of nulls for column loan_status: is 0.0
the percentage of nulls for column payment_plan: is 0.0
the percentage o

TypeError: DataFrame.insert() missing 2 required positional arguments: 'column' and 'value'